In [ ]:
# Cut of gradio section, should result in a simple inputsketch and input prompt pair 
import gradio as gr
import time

inputSketch = None
inputPrompt = ""



def sleep(im):
    time.sleep(5)
    return [im["background"], im["layers"][0], im["layers"][1], im["composite"]]

def predict(im):
    return im["composite"]

def submit(im, prompt):
    global inputSketch
    inputSketch = im
    global inputPrompt
    inputPrompt =  prompt
    return "prompt and sketch submited"

with gr.Blocks() as demo:
    with gr.Row():
        im = gr.ImageEditor(
            type="numpy",
            crop_size="1:1",
            height=512,  # Define sketch height
            width=512    # Define sketch width
        )
        im_preview = gr.Image()
    
    with gr.Row():
        userPrompt = gr.TextArea(label="input prompt:")

    with gr.Row():
        submit_button = gr.Button("submbit")

    submit_button.click(fn= submit, inputs=[im_preview, userPrompt])

    # n_upload = gr.Number(0, label="Number of upload events", step=1)
    # n_change = gr.Number(0, label="Number of change events", step=1)
    # n_input = gr.Number(0, label="Number of input events", step=1)

    # im.upload(lambda x: x + 1, outputs=n_upload, inputs=n_upload)
    # im.change(lambda x: x + 1, outputs=n_change, inputs=n_change)
    # im.input(lambda x: x + 1, outputs=n_input, inputs=n_input)
    im.change(predict, outputs=im_preview, inputs=im, show_progress="hidden")


    
if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", share=True)



print(inputSketch)
print(inputPrompt)

In [ ]:
# Section to test if the image is not empty and save them for later testing
from PIL import Image
# Quick empty check
import numpy as np
# print(inputSketch)
# print(inputPrompt)


countzero_in1 = not np.any(inputSketch)
print("Whole array contains zeroes in array1 ?: ", countzero_in1)

if countzero_in1 == False: 
    # Save the sketch as png
    sketch_image = Image.fromarray(inputSketch)
    sketch_image.save("saved_sketch_test.png")
    print("Sketch saved as 'saved_sketch.png'")
    
    with open("saved_prompt_test.txt", "w") as f:
        f.write(inputPrompt)
        print("Prompt saved as 'saved_prompt.txt'")
        


In [ ]:
prompts = [
    "A small iron sword.",
    "A large two-handed sword.",
    "A steel short sword.",
    "A bronze sword.",
    "A black sword with a red gem.",
    "A rusty old sword.",
    "A shining silver sword.",
    "A sword with a leather grip.",
    "A fire sword.",
    "A sword with an icy blue blade.",
    "A knight's sword with a golden hilt.",
    "A plain copper sword.",
    "A curved blade sword.",
    "A sword with a green handle.",
    "A glowing gold sword.",
    "A small dagger-like sword.",
    "A big heavy sword.",
    "A thin, fast sword.",
    "A red sword with jagged edges.",
    "A royal sword with a blue gem."
]

# Smaller subset for intial tests
prompts_subset = [
    "A small iron sword.",
    "A large two-handed sword.",
    "A steel short sword.",
    "A bronze sword.",
    "A black sword with a red gem."
]

In [ ]:
import random
import numpy as np
import torch
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel, AutoencoderKL
from diffusers import EulerAncestralDiscreteScheduler
from PIL import Image
import time
import os

# Base SDXL model
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"

# ControlNet model (Scribble SDXL)
controlnet_model_id = "xinsir/controlnet-scribble-sdxl-1.0"

# LoRA file (Pixel-Art-XL)
lora_file_path = "pixel-art-xl-v1.1.safetensors"

# VAE for SDXL
vae_id = "madebyollin/sdxl-vae-fp16-fix"

# Load ControlNet model
controlnet = ControlNetModel.from_pretrained(controlnet_model_id, torch_dtype=torch.float16)

# Load SDXL pipeline with ControlNet
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    base_model_id,
    controlnet=controlnet,
    vae=AutoencoderKL.from_pretrained(vae_id, torch_dtype=torch.float16),
    safety_checker=None,
    torch_dtype=torch.float16,
)

# Load LoRA weights
pipe.load_lora_weights(lora_file_path, adapter_name="pixel-art-xl")

# Enable GPU optimizations
pipe.to("cuda")

# Scheduler
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

# Image generation function with time tracking
def generate_pixel_art(prompts, sketch_path, output_dir, num_images=4, log_file="time_log.txt"):
    # Open the log file for writing
    with open(log_file, "a") as log:
        # Load and process the sketch
        if sketch_path:
            sketch = Image.open(sketch_path).convert("L")  # Convert to grayscale
        else:
            raise ValueError("Please provide a sketch path.")

        # Resize sketch to match SDXL resolution (1024x1024)
        width, height = sketch.size
        ratio = np.sqrt(1024.0 * 1024.0 / (width * height))
        new_width, new_height = int(width * ratio), int(height * ratio)
        sketch = sketch.resize((new_width, new_height))
    
    #loop through the prompt list to handel each prompt as a single unit is stead of the whole list
        for prompt in prompts:
            # Generate images
            for i in range(num_images):
                start_time = time.time()  # Start the timer

                # ControlNet conditioning scale - LOWER this to make the AI follow the sketch LESS
                controlnet_conditioning_scale = 0.5 # How much weight to assign to the conditioning inputs
                guidance_scale = 8.0  # Increase this to make it follow the prompt more
                num_inference_steps = 20  

                # Generate the image
                image = pipe(
                    prompt=prompt,
                    negative_prompt="lowres, blurry, bad anatomy, extra limbs",
                    image=sketch,
                    controlnet_conditioning_scale=controlnet_conditioning_scale,  
                    width=new_width,
                    height=new_height,
                    num_inference_steps=num_inference_steps, 
                    guidance_scale=guidance_scale, 
                ).images[0]


                # Stop the timer
                elapsed_time = time.time() - start_time

                # Convert to RGBA and add transparency
                image = image.convert("RGBA")
                data = np.array(image)
                r, g, b, _ = data.T
                white_areas = (r == 255) & (g == 255) & (b == 255)
                data[..., -1][white_areas] = 0  # Make white areas transparent
                image = Image.fromarray(data)

                # Save the output
                output_path = f"{output_dir}/{prompt}_image_{i + 1}.png"
                image.save(output_path)

                # Log the time taken
                log.write(f"Prompt: {prompt}, Image {i + 1}, Time: {elapsed_time:.2f} seconds\n")
                print(f"Generated image saved at: {output_path} (Time: {elapsed_time:.2f} seconds)")



# Path to the sketch
sketch_path = "your input sketch.png" 

# Directory to save outputs
output_dir = "generated_images"
os.makedirs(output_dir, exist_ok=True)

# Log file
log_file = "time_log.txt"
if os.path.exists(log_file):
    os.remove(log_file)  # Clear the log file if it already exists

# Generate pixel art for each prompt
for prompt in prompts:
    generate_pixel_art(prompts_subset_1, sketch_path, output_dir, log_file=log_file)


In [ ]:
# Background remover test with another model 
from rembg import remove
from PIL import Image

# Load your input image
input_path = 'AsmallIronSword3_second_run.png'
output_path = 'A_Small_Iron_Sword_3_second_NBG.png'

with Image.open(input_path) as img:
    # Remove the background
    img_no_bg = remove(img)
    # Save the output image
    img_no_bg.save(output_path)
